In [26]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [28]:
import os
import streamlit as st
from langchain_community.document_loaders import WebBaseLoader


# Charger la page web avec WebBaseLoader
loader = WebBaseLoader("https://orange.jobs/jobs/v3/offers/142429?lang=fr")

page_data = loader.load().pop().page_content
print(page_data)






orange.jobs - Stage - Chargé de Projet Marketing Expérience Client F/H - Orange Jobs













































Orange Jobs


Aller au contenu






				Accueil



Jobs

Chercher un job
CDI
CDD
Stage
Apprentissage
Thèse
VIE
Postdoc
Professionnalisation
Matchez votre CV
Créer mon profil




				Etapes de recrutement




				Rédiger mon CV





#LifeAtOrange



				Qui sommes-nous ?




				24h avec ...




				Apprendre




				Challenge




				Engagement




				Equilibre pro-perso




				Histoires vraies




				Innovation




				Les femmes dans la tech




				Nos bureaux




				Onboarding




				Rémunération




				Responsabilité sociétale





Métiers



				Cloud




				Conseil




				Cybersécurité




				Développement




				IA-Data




				IoT




				Recherche




				Relation client




				Réseau




				Santé




				Wholesale





Etudiants



				Graduate Program




				Stages




				Alternance




				CFA Orange




				VIE




				Thèses

In [29]:
#prompt template extracting content out of job description 
from langchain_core.prompts import PromptTemplate
import streamlit as st 

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and extract from section name of sender of email and not the person who provided the job ofeer and all details provided then return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` , `description` and `mentions`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
#getting prompt and pass it to llm
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
#store it to a response object
type(res.content)

str

In [30]:
#convert to json
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Stage - Chargé de Projet Marketing Expérience Client F/H',
  'experience': '6 mois',
  'skills': ["Sensibilité à l'expérience client",
   'Structuré, enthousiaste, bon communicant',
   'Travail en équipe',
   'Adaptabilité',
   'Maîtrise du pack office (notamment PowerPoint)',
   'Synthèse',
   'Expérience dans le conseil, les études, le benchmark (un atout)'],
  'description': "Vous intégrez une petite équipe de 5 personnes avec une forte cohésion et un esprit d'entraide. Vous contribuez au développement de l'activité de la Direction « Customer Experience Simplification » en enrichissant la proposition de valeur. Vous contribuez à des projets de transformation avec de forts enjeux business pour les filiales Orange.",
  'mentions': ['Danielle, maitre de stage',
   "Direction de la Simplification de l'Expérience Client",
   'Groupe Orange',
   'Orange Jobs']}]

In [31]:
type(json_res)

list

In [32]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [33]:
#insert prtfolio in chromadb, it should extract skills from job and match and retrieve 
import uuid
import chromadb
#using client will create chromadb in lelory but will use persistent client so create db on disk : will create db stored in a folder so i have it in a disk
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")
#if collection created for fisrt time 
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [34]:
links= collection.query(query_texts=["Machine Learning", "computer vision"], n_results=6).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [35]:
print(json_res)  # Check the structure of json_res


[{'role': 'Stage - Chargé de Projet Marketing Expérience Client F/H', 'experience': '6 mois', 'skills': ["Sensibilité à l'expérience client", 'Structuré, enthousiaste, bon communicant', 'Travail en équipe', 'Adaptabilité', 'Maîtrise du pack office (notamment PowerPoint)', 'Synthèse', 'Expérience dans le conseil, les études, le benchmark (un atout)'], 'description': "Vous intégrez une petite équipe de 5 personnes avec une forte cohésion et un esprit d'entraide. Vous contribuez au développement de l'activité de la Direction « Customer Experience Simplification » en enrichissant la proposition de valeur. Vous contribuez à des projets de transformation avec de forts enjeux business pour les filiales Orange.", 'mentions': ['Danielle, maitre de stage', "Direction de la Simplification de l'Expérience Client", 'Groupe Orange', 'Orange Jobs']}]


In [36]:
job = json_res[0]  # Assuming the relevant data is in the first element of the list
job.get('skills', [])  # Accessing the 'skills' and handle if 'skills' key is not present.

["Sensibilité à l'expérience client",
 'Structuré, enthousiaste, bon communicant',
 'Travail en équipe',
 'Adaptabilité',
 'Maîtrise du pack office (notamment PowerPoint)',
 'Synthèse',
 'Expérience dans le conseil, les études, le benchmark (un atout)']

In [37]:
job

{'role': 'Stage - Chargé de Projet Marketing Expérience Client F/H',
 'experience': '6 mois',
 'skills': ["Sensibilité à l'expérience client",
  'Structuré, enthousiaste, bon communicant',
  'Travail en équipe',
  'Adaptabilité',
  'Maîtrise du pack office (notamment PowerPoint)',
  'Synthèse',
  'Expérience dans le conseil, les études, le benchmark (un atout)'],
 'description': "Vous intégrez une petite équipe de 5 personnes avec une forte cohésion et un esprit d'entraide. Vous contribuez au développement de l'activité de la Direction « Customer Experience Simplification » en enrichissant la proposition de valeur. Vous contribuez à des projets de transformation avec de forts enjeux business pour les filiales Orange.",
 'mentions': ['Danielle, maitre de stage',
  "Direction de la Simplification de l'Expérience Client",
  'Groupe Orange',
  'Orange Jobs']}

In [ ]:
#prompt template that generate email 
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
       ### INSTRUCTION:
       write professionnal email according to job description you extracted .
       ### VALID JSON (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Candidature pour le Stage - Chargé de Projet Marketing Expérience Client F/H

Dear Danielle,

Je me permets de vous contacter suite à la publication de l'offre de stage "Chargé de Projet Marketing Expérience Client F/H" au sein de la Direction de la Simplification de l'Expérience Client du Groupe Orange. Je suis particulièrement intéressé par cette opportunité de rejoindre une équipe dynamique et motivée pour contribuer au développement de l'activité de la direction.

Avec mes compétences en marketing et ma sensibilité à l'expérience client, je suis convaincu que je peux apporter une valeur ajoutée à votre équipe. Je suis structuré, enthousiaste et bon communicant, ce qui me permet de travailler efficacement en équipe et de m'adapter rapidement aux nouvelles situations. De plus, j'ai une bonne maîtrise du pack office, notamment PowerPoint, ce qui me permet de présenter des idées de manière claire et concise.

Je suis également attiré par la possibilité de contribuer à des proj